# Direct and inverse geometry of 3d robots
This notebook introduces the kinematic tree of Pinocchio for a serial manipulator, explain how to compute the forward and inverse geometry (from configuration to end-effector placements, and inversely). The ideas are examplified with a simplified case-study taken from parallel robotics.

In [1]:
import magic_donotload

NB: as for all the tutorials, a magic command %do_not_load is introduced to hide the solutions to some questions. Change it for %load if you want to see (and execute) the solution.


## Set up
We will need Pinocchio, Gepetto-Viewer, SciPy for the solvers

In [2]:
import math
import time
from pinocchio.robot_wrapper import RobotWrapper
from pinocchio.utils import *
import pinocchio as pio
import eigenpy
from scipy.optimize import fmin_bfgs
eigenpy.switchToNumpyMatrix()

## Kinematic tree in Pinocchio
Let's now play with 3D robots. We will load the models from URDF files.

*The robot UR5* is a low-cost manipulator robot with good performances. It is a fixed robot with one 6-DOF arms developed by the Danish company Universal Robot. All its 6 joints are revolute joints. Its configuration is in R^6 and is not subject to any constraint. The model of UR5 is described in a URDF file, with the visuals of the bodies of the robot being described as meshed (i.e. polygon soups) using the Collada format ".dae". Both the URDF and the DAE files are available in the repository in the model directory. 

This robot model, as well as other models used in the notebooks, are installed from the apt paquet robotpkg-example-robot-data and stored in /opt/openrobots/share/example-robot-data.


In [3]:
# %load -r 9-14 tp2/simple_pick_and_place 
exampleRobotDataPath = '/opt/openrobots/share/example-robot-data/robots/'
urdf = exampleRobotDataPath + 'ur_description/urdf/ur5_gripper.urdf'
robot = RobotWrapper.BuildFromURDF( urdf, [ exampleRobotDataPath, ] )
robot.initViewer(loadModel=True)
NQ = robot.model.nq
NV = robot.model.nv

The kinematic tree is represented by two C++ objects called Model (which contains the model constants: lengths, masses, names, etc) and Data (which contains the working memory used by the model algorithms). Both C\++ objects are contained in a unique Python class. The first class is called RobotWrapper and is generic.

In [4]:
print(robot.model)

Nb joints = 7 (nq=6,nv=6)
  Joint 0 universe: parent=0
  Joint 1 shoulder_pan_joint: parent=0
  Joint 2 shoulder_lift_joint: parent=1
  Joint 3 elbow_joint: parent=2
  Joint 4 wrist_1_joint: parent=3
  Joint 5 wrist_2_joint: parent=4
  Joint 6 wrist_3_joint: parent=5



For the next steps, we are going to work with the RobotWrapper.

Import the class RobotWrapper and create an instance of this class in the python terminal. At initialization, RobotWrapper will read the model description in the URDF file given as argument. In the following, we will use the model of the UR5 robot, available in the directory "models" of pinocchio (available in the homedir of the VBox). The code of the RobotWrapper class is in /opt/openrobots/lib/python2.7/site-packages/pinocchio/robot_wrapper.py . Do not hesitate to have a look at it and to take inspiration from the implementation of the class functions.

Here are some import methods of the class.


* robot.q0 contains a reference initial configuration of the robot (not a pretty good one for the UR-5).
* robot.display(q) display the configuration q in the viewer.

In [5]:
robot.display(robot.q0)

* robot.index('joint name') returns the index of the joint.

In [6]:
robot.index(' wrist_3_joint')

7

* robot.model.names is a container (~list) that contains all the joint names

In [7]:
for i,n in enumerate(robot.model.names): print(i,n)

0 universe
1 shoulder_pan_joint
2 shoulder_lift_joint
3 elbow_joint
4 wrist_1_joint
5 wrist_2_joint
6 wrist_3_joint


* robot.model.frames contains all the import frames attached to the robot. 

In [8]:
for f in robot.model.frames: print(f.name,'attached to joint #',f.parent)

universe attached to joint # 0
root_joint attached to joint # 0
world attached to joint # 0
world_joint attached to joint # 0
base_link attached to joint # 0
base_link-base_fixed_joint attached to joint # 0
base attached to joint # 0
shoulder_pan_joint attached to joint # 1
shoulder_link attached to joint # 1
shoulder_lift_joint attached to joint # 2
upper_arm_link attached to joint # 2
elbow_joint attached to joint # 3
forearm_link attached to joint # 3
wrist_1_joint attached to joint # 4
wrist_1_link attached to joint # 4
wrist_2_joint attached to joint # 5
wrist_2_link attached to joint # 5
wrist_3_joint attached to joint # 6
wrist_3_link attached to joint # 6
ee_fixed_joint attached to joint # 6
ee_link attached to joint # 6
wrist_3_link-tool0_fixed_joint attached to joint # 6
tool0 attached to joint # 6


* robot.placement(idx) and robot.framePlacement(idx) returns the placement (i.e. translation+rotation of the joint / frame in argument.

In [9]:
robot.placement(robot.q0,6) # Placement of the end effector.

  R =
          -1            0  3.58979e-09
           0            1            0
-3.58979e-09            0           -1
  p =   0.81725   0.10915 -0.005491

## Display simple geometries
The robot is displayed in the viewer through robot.display(q). Other geometries (cubes, spheres, etc) can be displayed as well, see [the previous notebook](1_geometry_2d.ipynb#section_display_objects).

In [10]:
# Add a red sphere
gv = robot.viewer.gui
ballID = "world/ball"
rgbt = [1.0, 0.2, 0.2, 1.0]  # red-green-blue-transparency
gv.addSphere(ballID, 0.05, rgbt)  # id, dim1, dim2, dim3, color
# Place the ball at the position ( 0.5, 0.1, 0.2 )
q_ball = [0.6, -0.1, 0.3, 1, 0, 0, 0]
gv.applyConfiguration(ballID, q_ball)
gv.refresh()

## Forward (direct) geometry

First, let's do some forward geometry, i.e. use Pinocchio to compute where is the end effector knowning the robot configuration.

### Simple pick ...

Say we have a target at position [.5,.1,.2] and we would like the robot to grasp it.
First decide (by any way you want, e.g. trial and error) the configuration of the robot so that the end effector touches the ball.


In [ ]:
q = zero(NQ)  ### set the correct values here
q[0] = 0.
q[1] = 1.
q[2] = 0.
q[3] = 0.
q[4] = 0.
q[5] = 0.
robot.display(q)

### ... and simple place

At the reference position you built, the end effector placement can be obtained by calling 

In [ ]:
robot.placement(q,6).translation

Only the translation part of the placement has been selected. The rotation is free.

Now, choose any trajectory you want in the configuration space (it can be sinus-cosinus waves, polynomials, splines, straight lines). Make a for loop to display the robot at sampling positions along this trajectory. The function sleep can be used to slow down the loop.

In [ ]:
time.sleep(.1) # in second

At each instant of your loop, recompute the position of the ball and display it so that it always "sticks" to the robot end effector.

In [ ]:
q = np.matrix([-.1,-1.2,1.7,.5,-.1,0]).T
q_ball = [0.45, 0.2, 0.25, 1, 0, 0, 0]
# yes, this configuration is not so precisely on the ball
robot.display(q)
ball = np.matrix(q_ball[:3]).T
eff_to_ball = robot.placement(q,6).translation - ball
v = np.matrix([.10, .20, .30, .20, .1, -.05]).T

for i in range(1000):
        q += v*1e-2
        ball = robot.placement(q,6).translation + eff_to_ball
        gv.applyConfiguration(ballID,tuple(ball.flat)+(1,0,0,0))
        robot.display(q)
        time.sleep(.01)

### Pick and place in 3D

Say now that the object is a rectangle and not a sphere. Pick the object at a reference position with the rotation that is imposed, so that the end effector is aligned with one of the faces of the rectangle.


In [ ]:
# %load -r 16-24 tp2/simple_pick_and_place.py
# Add a red box
gv = robot.viewer.gui
boxID = "world/box"
rgbt = [1.0, 0.2, 0.2, 1.0]  # red-green-blue-transparency
gv.addBox(boxID, 0.05, 0.1, 0.1, rgbt)  # id, dim1, dim2, dim3, color
# Place the box at the position ( 0.5, 0.1, 0.2 )
q_box = [0.5, 0.1, 0.2, 1, 0, 0, 0]
gv.applyConfiguration(boxID, q_box)
gv.refresh()

A configuration with the arm nicely attached to the box is:

In [ ]:
# %load -r 31-38 tp2/simple_pick_and_place.py

# Configuration for picking the box
q = zero(NQ)
q[0] = -0.375
q[1] = -1.2
q[2] = 1.71
q[3] = -q[1] - q[2]
q[4] = q[0]

Redo the same question as before, but now also choosing the orientation of the box. For that, at each robot configuration in your for-loop, compute the box placement wrt the world (let's denote it by oMbox) and display both the box and the robot configuration in the view. You can transform a SE3 object oMbox into a tuple [X,Y,Z,Quaternion] with

In [ ]:
M = robot.placement(q,6)
pio.se3ToXYZQUATtuple(M)

In [ ]:
%do_not_load -r 48-69 tp2/simple_pick_and_place.py

## Inverse geometry

We only yet computed the forward geometry, i.e. from configurations to end-effector placement. Let's to the inverse map not.

### Inverse geometry in 3D

Let's now first control the position (i.e. translation only) of the end effector of a manipulator robot to a given position. For this first part, we will use the fixed serial-chain robot model.

Recall first that the position (3D) of the joint with index "i=6" at position "q" can be access by the following two lines of code.

In [ ]:
robot.placement(q,6).translation

Using the scipy solver [used in the previous notebook](1_geometry_2d.ipynb#section_optim), compute a configuration q where the end effector reaches p. For that, implement a cost function that takes a configuration as argument and returns the squared distance between the end effetor and the target.

In [ ]:
%load -r 35- tp2/invgeom3d.py

### Inverse geometry in 6D
6D means: translation and rotation. Change the previous cost function for a cost measuring the difference between the current placement root.placement(q,6) and a reference placement oMdes. 
For that, you can use the SE(3) log function to score the distance between two placements. The log returns a 6D velocity, represented by a class Motion, that must be transformed to a vector of R^6 from which you can take the norm.


In [ ]:
pio.log(pio.SE3.Identity()).vector

In [ ]:
%do_not_load -r 35- tp2/invgeom6d.py

For the next sections we will not need the quadruped any more: let's clean the viewer.

In [ ]:
gv.deleteNode('world',True)

### Optimizing in the quaternion space

Let's now work with a floating robot: the quadruped ANYmal. This robot has 12 joints, but Q-space of size 19 (robot.model.nq) and Q-tangent space of size 18 (robot.model.nv). This is because with need 7D vector to encode the robot placement in space, which indeed to only 6 DOF.

Run the following code. Can you explain what just happened? Then correct it to have a proper optimization of ANYmal configuration.

In [ ]:
# %load tp2/floating.py
'''
Stand-alone inverse geom in 3D.  Given a reference translation <target> ,
it computes the configuration of the UR5 so that the end-effector position (3D)
matches the target. This is done using BFGS solver. While iterating to compute
the optimal configuration, the script also display the successive candidate
solution, hence moving the robot from the initial guess configuaration to the
reference target.
'''

import time
import numpy as np
import pinocchio as pio
from example_robot_data import loadANYmal
from pinocchio.utils import *
from scipy.optimize import fmin_bfgs
import eigenpy
eigenpy.switchToNumpyMatrix()

# --- Load robot model
robot = loadANYmal()
robot.initViewer(loadModel=True)
NQ = robot.model.nq
NV = robot.model.nv
robot.display(robot.q0)

robot.feetIndexes = [ robot.model.getFrameId(frameName) for frameName in ['RH_FOOT','LH_FOOT','RF_FOOT','LF_FOOT' ] ]

# --- Add box to represent target
gv=robot.viewer.gui
gv.addSphere("world/red", .05, [1., .2, .2, .5])  # .1 is the radius
gv.addSphere("world/blue", .05, [.2, .2, 1., .5])  # .1 is the radius
gv.addSphere("world/green", .05, [.1, 1., .1, .5])  # .1 is the radius
gv.addSphere("world/purple", .05, [1., .2, 1., .5])  # .1 is the radius
#gv.addSphere("world/white", .05, [.9, .9, .9, .5])  # .1 is the radius

gv.addSphere("world/red_des", .05, [1., .2, .2, .95])  # .1 is the radius
gv.addSphere("world/blue_des", .05, [.2, .2, 1., .95])  # .1 is the radius
gv.addSphere("world/green_des", .05, [.1, 1., .1, .95])  # .1 is the radius
gv.addSphere("world/purple_des", .05, [1., .2, 1., .95])  # .1 is the radius
#gv.addSphere("world/white_des", .05, [.9, .9, .9, .95])  # .1 is the radius

#
# OPTIM 6D #########################################################
#

pdes = [
    np.matrix(  [[-0.7, -0.2, 1.2]]).T,
    np.matrix(  [[-0.3,  0.5,  0.8]]).T,
    np.matrix(  [[0.3, 0.1, -0.1]]).T,
    np.matrix(  [[0.9, 0.9, 0.5]]).T
    ]
for i in range(4): pdes[i][2]+=1
colors = ['red','blue','green','purple']

def cost(q):
    '''Compute score from a configuration'''
    q = np.matrix(q).T
    cost = 0.
    for i in range(4):
        p_i = robot.framePlacement(q, robot.feetIndexes[i]).translation
        cost += np.linalg.norm(p_i-pdes[i])**2
    return cost

def callback(q):
    q = np.matrix(q).T
    gv.applyConfiguration('world/box', pio.se3ToXYZQUATtuple(Mtarget))

    for i in range(4):
        p_i = robot.framePlacement(q, robot.feetIndexes[i])
        gv.applyConfiguration('world/'+colors[i], pio.se3ToXYZQUATtuple(p_i))
        gv.applyConfiguration('world/%s_des'%colors[i], tuple(pdes[i].flat)+(0,0,0,1))

    robot.display(q)
    time.sleep(1e-1)

Mtarget = pio.SE3(rotate('x',3.14/4), np.matrix([0.5, 0.1, 0.2]).T)  # x,y,z
qopt = fmin_bfgs(cost, robot.q0, callback=callback)
qopt = np.matrix(qopt).T



## Configuration of parallel robots
A parallel robot is composed of several kinematic chains (called the robot legs) that are all attached to the same end effector. This imposes strict constraints in the configuration space of the robot: a configuration is valide iff all the legs meets the same end-effector placement. We consider here only the geometry aspect of parallel robots (additionnally, some joints are not actuated, which causes additional problems).

The kinematic structure of a paralel robot indeed induces loops in the joint connection graph. In Pinocchio, we can only represents (one of) the underlying kinematic tree. The loop constraints have to be handled separately. An example that loads 4 manipulator arms is given below.

In [ ]:
%load tp2/example_parallel.py

 Each leg i (for i=0,1,2,3) of the robot is loaded in the list robots[i]. The loop constraints are that the relative placement of every leg end-effector must stay the same that in the initial configuration given as example in the above file.

Consider now that the orientation of the tool plate is given by the quaternion Quaternion(0.7,0.2,0.2,0.6), with the translation that you like (see [the notebook about rotations if you need more details](appendix1_quaternions.ipynb)). **Find using the above optimization routines the configuration of each robot leg so that the loop constraints are all met**.

